# Points Coordinate Converter & KMZ Creator

Supports **PNEZD** (Point, Northing, Easting, Elevation, Description) and **PENZD** (Point, Easting, Northing, Elevation, Description).

**Outputs:**
- points.csv
- points.kmz

In [2]:
import os
import pandas as pd
from pyproj import Transformer, CRS
from zipfile import ZipFile, ZIP_DEFLATED
from pyproj import CRS

In [ ]:
# ----------------------------
# Configuration
# ----------------------------
DIR = r"...\SW Geomatics Projects - US0050682.5847 - NTUA Ganado Lagoon Improvem\CAD\TOPO\XSURVEY\2019_Points"
file_name = "points_read.csv"       # Headerless, Comma-separated
input_format = "PNEZD"              # "PNEZD" or "PENZD"

in_crs  = "EPSG:6407"               # e.g., NAD83(2011) / Arizona East (International foot)
out_crs = "EPSG:4326"               # WGS84 lat/lon
altitude_mode = "absolute"          # 'absolute' | 'clampToGround' | 'relativeToGround'

# Optional: override vertical unit if different from XY (NOT COMMON, but possible)
# choices: None | 'us_survey_foot' | 'international_foot' | 'meter'
override_vertical_unit = None

# Build input path
input_path = os.path.join(DIR, file_name)

# Outputs
output_csv = os.path.join(DIR, f"points.csv")
kmz_path   = os.path.join(DIR, "points.kmz")

print(input_path)
print(output_csv)


IN CRS = EPSG:6407
OUT CRS = EPSG:4326
...\SW Geomatics Projects - US0050682.5847 - NTUA Ganado Lagoon Improvem\CAD\TOPO\XSURVEY\2019_Points\points_read.csv
...\SW Geomatics Projects - US0050682.5847 - NTUA Ganado Lagoon Improvem\CAD\TOPO\XSURVEY\2019_Points\points.csv


In [ ]:
# ----------------------------
# Helpers
# ----------------------------
def unit_factor_from_epsg(crs_code: str) -> float:
    """
    Return scale factor to convert horizontal axis units to meters.
    Uses pyproj CRS metadata instead of hardcoding EPSG mappings.
    """
    try:
        crs = CRS.from_user_input(crs_code)
        # Get the first axis (usually horizontal) unit name and conversion factor
        axis_info = crs.axis_info[0]
        unit_name = axis_info.unit_name.lower()  # e.g., 'foot', 'metre'
        print("UoM: ", unit_name)
        conversion_factor = axis_info.unit_conversion_factor  # factor to meters
        return conversion_factor
    except Exception:
        # Default to meters if CRS cannot be parsed
        return 1.0

def unit_factor_from_override(name):
    if name is None: return None
    name = name.lower()
    if name == "us_survey_foot":     return 0.3048006096012192
    if name == "international_foot": return 0.3048
    if name == "meter":              return 1.0
    raise ValueError("override_vertical_unit must be: None | 'us_survey_foot' | 'international_foot' | 'meter'")

def esc(s: str) -> str:
    return (s or "").replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")

In [6]:
# ----------------------------
# Read CSV
# ----------------------------
# Expect 5 columns by position:
#   PNEZD: Point, Northing, Easting, Elevation, Description
#   PENZD: Point, Easting, Northing, Elevation, Description

print("IN CRS =", in_crs)
print("OUT CRS =", out_crs)

df = pd.read_csv(input_path, header=None, delimiter=",")

fmt = input_format.upper()
if fmt == "PNEZD":
    df.columns = ["PID", "Northing", "Easting", "Elevation", "Description"]
elif fmt == "PENZD":
    df.columns = ["PID", "Easting", "Northing", "Elevation", "Description"]
else:
    raise ValueError("input_format must be 'PNEZD' or 'PENZD'")

# Ensure numeric (coerce bad values to NaN)
for c in ["Easting", "Northing", "Elevation"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ----------------------------
# Determine Z→meters factor
# ----------------------------
override_factor = unit_factor_from_override(override_vertical_unit)
vertical_to_meter = override_factor if override_factor is not None else unit_factor_from_epsg(in_crs)
df["Elevation_m"] = df["Elevation"] * vertical_to_meter

# ----------------------------
# Transform coordinates (uses Elevation_m)
# ----------------------------
transformer = Transformer.from_crs(in_crs, out_crs, always_xy=True)
lon, lat, h = transformer.transform(
    df["Easting"].values, df["Northing"].values, df["Elevation_m"].values
)
df["Longitude"] = lon
df["Latitude"]  = lat
df["EllipsoidalHeight_m"] = h

# ----------------------------
# Write CSV
# ----------------------------
# Keep original fields + transformed outputs in meters
df.to_csv(output_csv, index=False)

# ----------------------------
# Build KML content (Name = PID, Description = Description)
# ----------------------------
kml_header = (
    '<?xml version="1.0" encoding="UTF-8"?>\n'
    '<kml xmlns="http://www.opengis.net/kml/2.2">\n'
    '<Document>\n'
)
kml_footer = "</Document>\n</kml>\n"

placemarks = []
for _, row in df.iterrows():
    name = esc(str(row.get("PID", "")))       # PID as Name
    desc = esc(str(row.get("Description", "")))   # Description as Description
    lon = row["Longitude"]
    lat = row["Latitude"]
    alt = row["EllipsoidalHeight_m"] if pd.notna(row["EllipsoidalHeight_m"]) else 0
    pm = (
        "  <Placemark>\n"
        f"    <name>{name}</name>\n"
        f"    <description>{desc}</description>\n"
        "    <Point>\n"
        f"      <coordinates>{lon},{lat},{alt}</coordinates>\n"
        f"      <altitudeMode>{altitude_mode}</altitudeMode>\n"
        "    </Point>\n"
        "  </Placemark>\n"
    )
    placemarks.append(pm)

kml_text = kml_header + "".join(placemarks) + kml_footer

# ----------------------------
# Write KMZ
# ----------------------------
with ZipFile(kmz_path, "w", ZIP_DEFLATED) as z:
    z.writestr("doc.kml", kml_text.encode("utf-8"))

print(f"Wrote:\n  CSV: {output_csv}\n  KMZ: {kmz_path}")
df.head()


IN CRS = EPSG:6407
OUT CRS = EPSG:4326


FileNotFoundError: [Errno 2] No such file or directory: '...\\SW Geomatics Projects - US0050682.5847 - NTUA Ganado Lagoon Improvem\\CAD\\TOPO\\XSURVEY\\2019_Points\\points_read.csv'